🔧  **Setup Required**: Before running this notebook, please follow the [setup instructions](../README.md#setup-instructions) to configure your environment and API keys. **You will need to ensure you've executed the Indexing pipeline before completing this exercise**

# Systematic RAG Evaluation: Naive vs Hybrid Comparison

## Overview

This notebook demonstrates a **systematic evaluation workflow** for comparing two RAG (Retrieval-Augmented Generation) approaches using **Haystack custom components**. We'll create a reproducible pipeline to:

1. **Load evaluation datasets** from CSV files
2. **Process queries** through both Naive and Hybrid RAG SuperComponents 
3. **Generate comprehensive metrics** using the RAGAS framework
4. **Compare performance** systematically

## Learning Objectives

By the end of this notebook, you will understand how to:
- Create reusable evaluation components for RAG systems
- Build scalable pipelines for systematic RAG comparison
- Interpret RAGAS metrics in comparative context
- Make data-driven decisions between RAG approaches

## Evaluation Pipeline

Our pipeline consists of three main components:

```
CSV Data → RAGDataAugmenter → RagasEvaluation → Metrics & Results
```

**Key Benefits:**
- **Systematic**: Same evaluation conditions for both RAG systems
- **Reproducible**: Consistent evaluation across experiments
- **Scalable**: Easy to add new RAG implementations
- **Comprehensive**: Multiple metrics provide complete assessment

---

## Component 1: CSV Data Loader

The **CSVReaderComponent** serves as the entry point for our evaluation pipeline. It handles loading synthetic evaluation datasets and ensures data quality before processing.

**Key Features:**
- **Robust Error Handling**: Validates file existence and data integrity
- **Pandas Integration**: Returns data as DataFrame for easy manipulation
- **Pipeline Compatible**: Designed to work seamlessly with Haystack pipelines

**Input:** File path to CSV containing evaluation queries and ground truth
**Output:** Pandas DataFrame ready for RAG processing

In [ ]:
import pandas as pd
from pathlib import Path
from haystack import component
from typing import Optional, Dict, Any, Union
import logging

logger = logging.getLogger(__name__)

@component
class CSVReaderComponent:
    """Reads a CSV file into a Pandas DataFrame."""

    @component.output_types(data_frame=pd.DataFrame)
    def run(self, source: Union[str, Path]):
        """
        Reads the CSV file from the first source in the list.
        
        Args:
            sources: List of file paths to CSV files. Only the first file will be processed.
            
        Returns:
            dict: Dictionary containing the loaded DataFrame under 'data_frame' key.
            
        Raises:
            FileNotFoundError: If the file doesn't exist or can't be read.
            ValueError: If the DataFrame is empty after loading.
        """
        if not source:
            raise ValueError("No sources provided")
            

        try:
            df = pd.read_csv(source)
        except FileNotFoundError:
            raise FileNotFoundError(f"File not found at {source}")
        except Exception as e:
            raise ValueError(f"Error reading CSV file {source}: {str(e)}")

        # Check if DataFrame is empty using proper pandas method
        if df.empty:
            raise ValueError(f"DataFrame is empty after loading from {source}")

        return {"data_frame": df}

## Component 2: Async RAG Data Augmentation

The **AsyncRAGDataAugmenterComponent** is the core of our evaluation workflow. It processes queries through a RAG SuperComponent with concurrent execution for optimal performance.

**Key Features:**

1. **Concurrent Processing**: Processes multiple queries in parallel batches
2. **SuperComponent Flexibility**: Accepts any pre-configured RAG SuperComponent (Naive, Hybrid, or custom)
3. **Configurable Batch Size**: Control concurrency based on API rate limits
4. **Progress Tracking**: Real-time visibility into processing status
5. **Error Handling**: Gracefully handles failures without stopping the entire evaluation

**Performance Benefits:**
- **Speed**: Up to N× faster than sequential processing (where N = batch_size)
- **Scalability**: Efficiently handles large evaluation datasets
- **Resource Optimization**: Maximizes API call efficiency

**Pipeline Integration:**
- **Input**: DataFrame with queries from CSVReaderComponent  
- **Process**: Runs queries through RAG SuperComponent in concurrent batches
- **Output**: Augmented DataFrame with responses and retrieved contexts

**Why Async?**
- **Faster Iteration**: Quickly evaluate large datasets
- **Better Resource Utilization**: Maximize throughput without overwhelming APIs
- **Production Ready**: Scalable approach for continuous evaluation

## Implementation: Concurrent Query Processing

Below is the implementation of the async component that enables concurrent query processing.

### ⚠️ Important Thread-Safety Considerations

**RAG SuperComponent Sharing:**
- The `AsyncRAGDataAugmenterComponent` processes queries sequentially within each RAG system
- **Haystack SuperComponents are NOT guaranteed to be thread-safe**
- `batch_size` is fixed at 1 to ensure safe operation with shared component state

**Current Safety Approach:**
- **Fixed `batch_size=1`**: Processes queries sequentially within each system to avoid conflicts
- **Concurrent System Evaluation**: Multiple RAG systems can still be evaluated in parallel
- **Trade-off**: Sacrifices within-system concurrency for stability and reliability

**Why Sequential Processing:**
- Embedding models in RAG SuperComponents may share state
- Concurrent access to the same SuperComponent instance can cause race conditions
- Sequential processing within each system ensures consistent, reliable results

**Performance:**
- **2× speedup** from evaluating multiple RAG systems concurrently
- Safe and stable for production use
- No risk of embedding model conflicts or race conditions

In [ ]:
import asyncio
from typing import List
from haystack import SuperComponent
import traceback
import logging

logger = logging.getLogger(__name__)


@component
class AsyncRAGDataAugmenterComponent:
    """
    Async version of RAGDataAugmenterComponent for concurrent system evaluation.
    
    Key Features:
    - Processes queries sequentially within each RAG system (thread-safe)
    - Enables concurrent evaluation of multiple RAG systems
    - Progress tracking for long-running evaluations
    - Robust error handling
    
    Thread-Safety Design:
    - Fixed batch_size=1 to avoid concurrent access to shared component state
    - Haystack SuperComponents are NOT guaranteed to be thread-safe
    - Sequential processing ensures no race conditions or model conflicts
    - Multiple systems can still be evaluated concurrently (2× speedup)
    
    Component Design:
    - run() method is synchronous (required by Haystack component interface)
    - Uses asyncio.run() internally to execute async processing
    - AsyncPipeline orchestrates concurrent execution across components
    """

    def __init__(self, rag_supercomponent: SuperComponent):
        """
        Initialize the Async RAG Data Augmenter.
        
        Args:
            rag_supercomponent: Pre-initialized RAG SuperComponent
        
        Note:
            batch_size is fixed at 1 to ensure thread-safe operation.
            This prevents race conditions when accessing the RAG SuperComponent,
            which may contain shared embedding models or other stateful components.
        """
        self.rag_supercomponent = rag_supercomponent
        self.batch_size = 1  # Fixed at 1 for thread safety
        self.output_names = ["augmented_data_frame"]

    async def _process_single_query(self, query: str, index: int) -> tuple:
        """
        Process a single query through the RAG SuperComponent.
        
        Thread Safety:
        - Uses asyncio.to_thread() to run synchronous RAG code in thread pool
        - Sequential processing (batch_size=1) prevents concurrent access
        
        Args:
            query: The query string to process
            index: The query index for tracking
            
        Returns:
            tuple: (index, answer, contexts)
        """
        try:
            # Run the RAG SuperComponent in a thread pool
            # asyncio.to_thread automatically handles thread pool execution
            rag_output = await asyncio.to_thread(
                self.rag_supercomponent.run, 
                query=query
            )
            
            # Extract answer and contexts
            answer = rag_output.get('replies', [''])[0]
            retrieved_docs = rag_output.get('documents', [])
            retrieved_contexts = [doc.content for doc in retrieved_docs]
            
            return (index, answer, retrieved_contexts)
        except Exception as e:
            logger.error(f"Error processing query {index} ('{query[:50]}...'): {str(e)}")
            traceback.print_exc()
            return (index, "", [])

    async def _process_batch(self, queries_with_indices: List[tuple]) -> List[tuple]:
        """
        Process a batch of queries.
        
        With batch_size=1, this processes one query at a time (thread-safe).
        
        Args:
            queries_with_indices: List of (index, query) tuples
            
        Returns:
            List of (index, answer, contexts) tuples
        """
        tasks = [
            self._process_single_query(query, idx) 
            for idx, query in queries_with_indices
        ]
        return await asyncio.gather(*tasks)

    async def _async_process(self, data_frame: pd.DataFrame) -> dict:
        """
        Internal async processing logic.
        
        This method contains the actual async processing logic and is called
        by the synchronous run() method via asyncio.run().
        
        Args:
            data_frame: DataFrame with 'user_input' column containing queries
            
        Returns:
            dict: Dictionary with augmented DataFrame containing responses and contexts
        """
        total_queries = len(data_frame)
        
        # Prepare queries with their indices
        queries_with_indices = list(enumerate(data_frame["user_input"].tolist()))
        
        # Initialize results storage
        results = [None] * total_queries
        
        # Process in batches (batch_size=1, so one at a time)
        for batch_start in range(0, total_queries, self.batch_size):
            batch_end = min(batch_start + self.batch_size, total_queries)
            batch = queries_with_indices[batch_start:batch_end]
            
            # Process batch (single query with batch_size=1)
            batch_results = await self._process_batch(batch)
            
            # Store results in correct order
            for idx, answer, contexts in batch_results:
                results[idx] = (answer, contexts)
        
        # Extract answers and contexts from results
        answers = [r[0] for r in results]
        contexts = [r[1] for r in results]
        
        # Create a copy to avoid modifying the original DataFrame
        # This ensures the component doesn't have side effects on input data
        data_frame_copy = data_frame.copy()
        data_frame_copy['response'] = answers
        data_frame_copy['retrieved_contexts'] = contexts
        
        logger.info(f"✓ Async RAG processing complete for {total_queries} queries!")
        return {"augmented_data_frame": data_frame_copy}

        return {"augmented_data_frame": data_frame_copy}

    @component.output_types(augmented_data_frame=pd.DataFrame)
    def run(self, data_frame: pd.DataFrame) -> dict:
        """
        Synchronous run method required by Haystack component interface.
        
        This method is synchronous and always returns a dictionary directly,
        maintaining consistency with the Haystack component contract.
        AsyncPipeline handles the orchestration of concurrent execution.
        
        Data Safety:
        - Creates a copy of input DataFrame to avoid mutation
        - Results stored in new columns on the copied DataFrame
        - Original data_frame parameter remains unchanged
        
        Thread Safety:
        - Sequential processing (batch_size=1) ensures thread safety
        - No concurrent access to the RAG SuperComponent instance
        - Safe for use with shared embedding models
        
        Args:
            data_frame: DataFrame with 'user_input' column containing queries
            
        Returns:
            dict: Dictionary with augmented DataFrame containing responses and contexts
        """
        # Use asyncio.run() to execute async processing within synchronous method

## Component 3: RAGAS Evaluation Engine

The **RagasEvaluationComponent** integrates the RAGAS framework into our Haystack pipeline, providing systematic evaluation metrics for RAG systems.

**Core Evaluation Metrics:**

| Metric | Purpose | What It Measures |
|--------|---------|------------------|
| **Faithfulness** | Response Quality | Factual consistency with retrieved context |
| **ResponseRelevancy** | Relevance | How well responses answer the questions |
| **LLMContextRecall** | Retrieval Quality | How well retrieval captures relevant information |
| **FactualCorrectness** | Accuracy | Correctness of factual claims in responses |

**Technical Implementation:**
- **Focused Metrics**: Core metrics for reliable comparison
- **LLM Integration**: Uses OpenAI GPT models for evaluation judgments  
- **Data Format Handling**: Automatically formats data for RAGAS requirements
- **Comprehensive Output**: Returns both aggregated metrics and detailed per-query results

In [ ]:
from ragas import EvaluationDataset, evaluate
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, ResponseRelevancy
from ragas.llms import HaystackLLMWrapper
import logging

logger = logging.getLogger(__name__)


@component
class RagasEvaluationComponent:
    """
    Integrates the RAGAS framework into Haystack pipeline for systematic evaluation.
    
    This component provides systematic evaluation metrics for RAG systems using
    the RAGAS framework with focus on core metrics for reliable comparison.
    
    Core Evaluation Metrics:
    - Faithfulness: Factual consistency with retrieved context
    - ResponseRelevancy: How well responses answer the questions  
    - LLMContextRecall: How well retrieval captures relevant information
    - FactualCorrectness: Correctness of factual claims in responses
    
    Technical Features:
    - Focused Metrics: Core metrics for reliable comparison
    - LLM Integration: Uses provided generator for evaluation judgments
    - Data Format Handling: Automatically formats data for RAGAS requirements
    - Comprehensive Output: Returns both aggregated metrics and detailed per-query results
    """
    
    def __init__(self, 
                 generator: Any,
                 metrics: Optional[List[Any]] = None):
        """
        Initialize the RAGAS Evaluation Component.
        
        Args:
            generator: LLM generator instance (e.g., OpenAIGenerator or OllamaGenerator).
            metrics: List of RAGAS metrics to evaluate (defaults to core metrics)
        """
        
        # Default to core metrics for systematic comparison
        if metrics is None:
            self.metrics = [
                Faithfulness(), 
                ResponseRelevancy(),
                LLMContextRecall(),
                FactualCorrectness()
            ]
        else:
            self.metrics = metrics
        
        # Configure RAGAS LLM for evaluation
        self.ragas_llm = HaystackLLMWrapper(generator)

    @component.output_types(metrics=Dict[str, float], evaluation_df=pd.DataFrame)
    def run(self, augmented_data_frame: pd.DataFrame):
        """
        Run RAGAS evaluation on augmented dataset.
        
        Args:
            augmented_data_frame: DataFrame with RAG responses and retrieved contexts
            
        Returns:
            dict: Dictionary containing evaluation metrics and detailed results DataFrame
        """
        
        # 1. Map columns to Ragas requirements
        ragas_data = pd.DataFrame({
            'user_input': augmented_data_frame['user_input'],
            'response': augmented_data_frame['response'], 
            'retrieved_contexts': augmented_data_frame['retrieved_contexts'],
            'reference': augmented_data_frame['reference'],
            'reference_contexts': augmented_data_frame['reference_contexts'].apply(eval)
        })

        # 2. Create EvaluationDataset
        dataset = EvaluationDataset.from_pandas(ragas_data)
        
        # 3. Run Ragas Evaluation
        results = evaluate(
            dataset=dataset,
            metrics=self.metrics,
            llm=self.ragas_llm
        )
        
        results_df = results.to_pandas()
        
        return {"metrics": results, "evaluation_df": results_df}

---

# Systematic RAG Comparison: Naive vs Hybrid

Now we'll systematically evaluate both RAG SuperComponents using the same evaluation pipeline. This ensures fair comparison with identical evaluation conditions.

## Comparison Strategy

Our approach enables **systematic comparison**:

1. **Same Dataset**: Both systems evaluated on identical test queries
2. **Same Metrics**: Consistent evaluation criteria across both approaches
3. **Same Pipeline**: Identical processing workflow eliminates bias
4. **Reproducible Results**: Pipeline ensures consistent evaluation conditions

## Dataset Information

We'll use a synthetic evaluation dataset:
- **`synthetic_tests_advanced_branching_10.csv`**: Focused dataset for comparison

**Dataset Structure:**
- `user_input`: Questions to ask the RAG system
- `reference`: Ground truth answers for comparison
- `reference_contexts`: Expected retrieved contexts

### Setup: Initialize Both RAG SuperComponents

First, we'll initialize both RAG SuperComponents with consistent parameters for fair comparison.

**Configuration:**
- **Same base parameters**: Both systems use identical core settings
- **Document store**: Shared Elasticsearch document store
- **Models**: Consistent LLM and embedding models for both systems

In [4]:
# --- Setup Environment & Dependencies ---
from scripts.rag.hybridrag import HybridRAGSuperComponent
from scripts.rag.naiverag import NaiveRAGSuperComponent
from haystack_integrations.document_stores.elasticsearch import ElasticsearchDocumentStore
import os
import logging

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# Initialize document store
document_store = ElasticsearchDocumentStore(hosts="http://localhost:9200")

# Create both RAG SuperComponents with base parameters for fair comparison
naive_rag_sc = NaiveRAGSuperComponent(
    document_store=document_store
)

hybrid_rag_sc = HybridRAGSuperComponent(
    document_store=document_store
)

print("Both RAG SuperComponents initialized successfully!")
print(f"Naive RAG: {naive_rag_sc.__class__.__name__}")
print(f"Hybrid RAG: {hybrid_rag_sc.__class__.__name__}")

Both RAG SuperComponents initialized successfully!
Naive RAG: NaiveRAGSuperComponent
Hybrid RAG: HybridRAGSuperComponent


## Evaluation Pipeline: Using Haystack AsyncPipeline

Now we'll create an evaluation pipeline using Haystack's `AsyncPipeline` for proper concurrent execution. This provides a cleaner, more maintainable approach than manually coordinating async components.

In [ ]:
from haystack import AsyncPipeline
import time
import logging

logger = logging.getLogger(__name__)

def create_evaluation_pipeline(
    rag_supercomponent,
    generator: Any
) -> AsyncPipeline:
    """
    Create an evaluation pipeline using Haystack's AsyncPipeline.
    
    AsyncPipeline handles concurrent execution of components automatically.
    Components themselves use synchronous run() methods, and AsyncPipeline
    orchestrates their async execution.
    
    Args:
        rag_supercomponent: The RAG system to evaluate
        generator: LLM generator for RAGAS evaluation
        
    Returns:
        AsyncPipeline: Configured evaluation pipeline
    
    Note:
        The AsyncRAGDataAugmenterComponent uses batch_size=1 internally
        for thread-safe operation. Concurrent evaluation happens at the
        system level (multiple RAG systems evaluated simultaneously).
    """
    pipeline = AsyncPipeline()
    
    # Add components to pipeline
    pipeline.add_component("reader", CSVReaderComponent())
    pipeline.add_component(
        "augmenter",
        AsyncRAGDataAugmenterComponent(
            rag_supercomponent=rag_supercomponent
        )
    )
    pipeline.add_component("evaluator", RagasEvaluationComponent(generator=generator))
    
    # Connect components
    pipeline.connect("reader.data_frame", "augmenter.data_frame")
    pipeline.connect("augmenter.augmented_data_frame", "evaluator.augmented_data_frame")
    
    return pipeline


async def evaluate_rag_system_async(
    rag_supercomponent,
    system_name: str,
    csv_file_path: str,
    generator: Any
) -> dict:
    """
    Asynchronously evaluate a single RAG system using AsyncPipeline.
    
    Args:
        rag_supercomponent: The RAG system to evaluate
        system_name: Name for logging
        csv_file_path: Path to evaluation dataset
        generator: LLM generator for RAGAS evaluation
        
    Returns:
        dict: Evaluation results with metrics and detailed DataFrame
    
    Note:
        Queries are processed sequentially within each system (batch_size=1)
        for thread safety. Multiple systems can be evaluated concurrently.
    """
    logger.info(f"Evaluating {system_name}...")
    
    # Create evaluation pipeline
    eval_pipeline = create_evaluation_pipeline(
        rag_supercomponent=rag_supercomponent,
        generator=generator
    )
    
    # Run pipeline asynchronously
    results = await eval_pipeline.run_async(data={"reader": {"source": csv_file_path}})
    
    return {
        "system_name": system_name,
        "metrics": results["evaluator"]["metrics"],
        "evaluation_df": results["evaluator"]["evaluation_df"]
    }


async def evaluate_multiple_rag_systems_async(
    rag_systems: List[tuple],
    csv_file_path: str
) -> List[dict]:
    """
    Evaluate multiple RAG systems concurrently using AsyncPipeline.
    
    This function enables concurrent evaluation of multiple RAG systems,
    providing a ~2× speedup compared to sequential evaluation.
    
    Args:
        rag_systems: List of (rag_supercomponent, system_name, generator) tuples
        csv_file_path: Path to evaluation dataset
        
    Returns:
        List of evaluation results for each system
    
    Performance:
        - Queries within each system: Sequential (batch_size=1, thread-safe)
        - Multiple systems: Concurrent (2× speedup for 2 systems)
        - Total speedup: ~2× for evaluating 2 systems simultaneously
    """
    logger.info(f"\n{'='*80}")
    logger.info(f"🚀 CONCURRENT EVALUATION OF {len(rag_systems)} RAG SYSTEMS")
    logger.info(f"{'='*80}")
    logger.info(f"Starting concurrent evaluation of {len(rag_systems)} RAG systems...")
    
    # Create evaluation tasks for each system
    tasks = [
        evaluate_rag_system_async(
            rag_supercomponent=rag_sc,
            system_name=name,
            csv_file_path=csv_file_path,
            generator=generator
        )
        for rag_sc, name, generator in rag_systems
    ]
    
    # Run all evaluations concurrently
    start_time = time.time()
    results = await asyncio.gather(*tasks)
    elapsed = time.time() - start_time
    
    logger.info(f"Evaluation complete in {elapsed:.1f}s")
    results = await asyncio.gather(*tasks)
    return results

## Run Concurrent Evaluation

Now let's evaluate both Naive and Hybrid RAG systems **simultaneously** using concurrent evaluation.

**Performance Benefits:**
- **2× Faster**: Both systems evaluated at the same time
- **Identical Conditions**: Both evaluations start simultaneously
- **Thread-Safe**: Sequential processing within each system prevents race conditions
- **Efficient Resource Use**: Maximizes computational efficiency

**Architecture:**
- **Within System**: Queries processed sequentially (batch_size=1)
- **Across Systems**: Multiple RAG systems evaluated concurrently
- **Result**: ~2× speedup with guaranteed thread safety

In [6]:
# --- Setup for Concurrent Evaluation ---

# Create separate generators for each evaluation (components can't be shared)
from haystack.components.generators import OpenAIGenerator
from haystack.utils import Secret
import os

eval_generator_naive = OpenAIGenerator(
    model="gpt-4o-mini",
    api_key=Secret.from_token(os.getenv("OPENAI_API_KEY"))
)

eval_generator_hybrid = OpenAIGenerator(
    model="gpt-4o-mini",
    api_key=Secret.from_token(os.getenv("OPENAI_API_KEY"))
)

# Define evaluation dataset
csv_file_path = "data_for_eval/synthetic_tests_advanced_branching_10.csv"

# Prepare RAG systems for concurrent evaluation
# Note: Each system processes queries sequentially (batch_size=1 internally)
# but both systems run concurrently for ~2× speedup
rag_systems = [
    (naive_rag_sc, "Naive RAG", eval_generator_naive),
    (hybrid_rag_sc, "Hybrid RAG", eval_generator_hybrid)
]

# Run concurrent evaluation
all_results = await evaluate_multiple_rag_systems_async(
    rag_systems=rag_systems,
    csv_file_path=csv_file_path
)

# Extract results for each system
naive_results_async = all_results[0]
hybrid_results_async = all_results[1]

print("\n✓ Concurrent evaluation complete!")
print(f"Naive RAG: {len(naive_results_async['evaluation_df'])} queries evaluated")
print(f"Hybrid RAG: {len(hybrid_results_async['evaluation_df'])} queries evaluated")

2025-12-08 13:10:43,263 - __main__ - INFO - 
2025-12-08 13:10:43,264 - __main__ - INFO - 🚀 CONCURRENT EVALUATION OF 2 RAG SYSTEMS
2025-12-08 13:10:43,264 - __main__ - INFO - ================================================================================
2025-12-08 13:10:43,264 - __main__ - INFO - Dataset: data_for_eval/synthetic_tests_advanced_branching_10.csv
2025-12-08 13:10:43,264 - __main__ - INFO - Processing: Sequential within each system (thread-safe)
2025-12-08 13:10:43,264 - __main__ - INFO - Concurrency: 2 systems evaluated in parallel
2025-12-08 13:10:43,265 - __main__ - INFO - Systems: ['Naive RAG', 'Hybrid RAG']
2025-12-08 13:10:43,265 - __main__ - INFO - ================================================================================

2025-12-08 13:10:43,265 - __main__ - INFO - 
2025-12-08 13:10:43,265 - __main__ - INFO - Starting async evaluation of Naive RAG
2025-12-08 13:10:43,265 - __main__ - INFO - ====================================================================

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-08 13:10:46,918 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2025-12-08 13:10:46,926 - haystack.core.pipeline.pipeline - INFO - Running component retriever
2025-12-08 13:10:46,926 - haystack.core.pipeline.pipeline - INFO - Running component retriever
2025-12-08 13:10:46,941 - elastic_transport.transport - INFO - POST http://localhost:9200/default/_search [status:200 duration:0.014s]
2025-12-08 13:10:46,943 - haystack.core.pipeline.pipeline - INFO - Running component prompt_builder
2025-12-08 13:10:46,944 - haystack.core.pipeline.pipeline - INFO - Running component llm
2025-12-08 13:10:46,941 - elastic_transport.transport - INFO - POST http://localhost:9200/default/_search [status:200 duration:0.014s]
2025-12-08 13:10:46,943 - haystack.core.pipeline.pipeline - INFO - Running component prompt_builder
2025-12-08 13:10:46,944 - haystack.core.pipeline.pipeline - INFO - Running component llm
2025-12-08 13:10:47,498 - haystack.core.pipelin

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-08 13:10:50,520 - haystack.core.pipeline.pipeline - INFO - Running component prompt_builder
2025-12-08 13:10:50,521 - haystack.core.pipeline.pipeline - INFO - Running component llm
2025-12-08 13:10:50,521 - haystack.core.pipeline.pipeline - INFO - Running component llm
2025-12-08 13:10:50,798 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:10:50,807 - __main__ - INFO - Processing query 4 of 10...
2025-12-08 13:10:50,798 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:10:50,807 - __main__ - INFO - Processing query 4 of 10...
2025-12-08 13:10:50,808 - haystack.core.pipeline.pipeline - INFO - Running component text_embedder
2025-12-08 13:10:50,808 - haystack.core.pipeline.pipeline - INFO - Running component text_embedder
2025-12-08 13:10:50,984 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:10:

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

2025-12-08 13:11:30,239 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:11:30,240 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:11:30,240 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:11:30,275 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:11:30,279 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:11:30,275 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:11:30,279 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:11:30,325 - httpx - INFO - HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-12-08 13:11:30,325 - httpx 

Evaluating:   0%|          | 0/40 [00:00<?, ?it/s]

2025-12-08 13:11:38,536 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,537 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,538 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,542 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,543 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,544 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,537 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,538 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,542 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,543 - haystack.core.pipeline.async_pipeline - INFO - Running component llm
2025-12-08 13:11:38,544 - haystack.core.pipeline.a


✓ Concurrent evaluation complete!
Naive RAG: 10 queries evaluated
Hybrid RAG: 10 queries evaluated


### View Evaluation Results

Let's examine the results from our concurrent evaluation of both systems.

In [7]:
# Display results from concurrent evaluation
print("Naive RAG - Async Evaluation Results:")
print(f"Metrics: {naive_results_async['metrics']}\n")
display(naive_results_async['evaluation_df'].head())

print("\n" + "="*80 + "\n")

print("Hybrid RAG - Async Evaluation Results:")
print(f"Metrics: {hybrid_results_async['metrics']}\n")
display(hybrid_results_async['evaluation_df'].head())

Naive RAG - Async Evaluation Results:
Metrics: {'faithfulness': 0.8972, 'answer_relevancy': 0.8696, 'context_recall': 0.9100, 'factual_correctness(mode=f1)': 0.5140}



,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_relevancy,context_recall,factual_correctness(mode=f1)
0,What Alexa do in AI?,"[What is AI, how does it work and why are some...","[What is AI, how does it work and why are some...","Alexa, as a voice-controlled virtual assistant...",Alexa is a voice-controlled virtual assistant ...,0.700000,0.913990,1.0,0.63
1,What happened to the UnitedHealthcare CEO Bria...,"[- James Kurose, “Testimony before the House C...",[Why is AI controversial?\nWhile acknowledging...,I don't have enough information to answer.,The BBC reported that Apple's AI falsely told ...,1.000000,0.000000,1.0,0.00
2,What is the current stance of the UK governmen...,"[What is AI, how does it work and why are some...",[Are there laws governing AI?\nSome government...,The current stance of the UK government regard...,"In the UK, Prime Minister Sir Keir Starmer has...",1.000000,0.976753,1.0,0.42
3,How does the GDPR impact the transparency and ...,[Smart cities\nMetropolitan governments are us...,[<1-hop>\n\nAI ethics and transparency\nAlgori...,The General Data Protection Regulation (GDPR) ...,The GDPR impacts the transparency and accounta...,0.615385,0.968926,1.0,0.55
4,What are the trends in the usage of ChatGPT fo...,[The yellow line represents the first cohort o...,[<1-hop>\n\n37% of messages are work-related f...,The usage of ChatGPT for Technical Help has sh...,The usage of ChatGPT for Technical Help has sh...,0.923077,0.981060,0.6,0.92




Hybrid RAG - Async Evaluation Results:
Metrics: {'faithfulness': 0.8636, 'answer_relevancy': 0.8679, 'context_recall': 0.9267, 'factual_correctness(mode=f1)': 0.5940}



,user_input,retrieved_contexts,reference_contexts,response,reference,faithfulness,answer_relevancy,context_recall,factual_correctness(mode=f1)
0,What Alexa do in AI?,"[What is AI, how does it work and why are some...","[What is AI, how does it work and why are some...",Alexa is a voice-controlled virtual assistant ...,Alexa is a voice-controlled virtual assistant ...,1.0,0.917891,1.0,0.67
1,What happened to the UnitedHealthcare CEO Bria...,[AI will reconfigure how society and the econo...,[Why is AI controversial?\nWhile acknowledging...,I don't have enough information to answer.,The BBC reported that Apple's AI falsely told ...,0.0,0.000000,1.0,0.00
2,What is the current stance of the UK governmen...,[AI will reconfigure how society and the econo...,[Are there laws governing AI?\nSome government...,The current stance of the UK government regard...,"In the UK, Prime Minister Sir Keir Starmer has...",1.0,0.979209,1.0,0.65
3,How does the GDPR impact the transparency and ...,"[- James Kurose, “Testimony before the House C...",[<1-hop>\n\nAI ethics and transparency\nAlgori...,The GDPR (General Data Protection Regulation) ...,The GDPR impacts the transparency and accounta...,1.0,0.971650,1.0,0.53
4,What are the trends in the usage of ChatGPT fo...,[37% of messages are work-related\nfor users w...,[<1-hop>\n\n37% of messages are work-related f...,The trends in the usage of ChatGPT for Technic...,The usage of ChatGPT for Technical Help has sh...,1.0,0.963631,0.8,0.76


## Side-by-Side Performance Comparison

Let's create a comprehensive comparison of both RAG systems to understand their relative performance across all metrics.

In [8]:
# Update the comparison to use async results
import pandas as pd

# Extract average metrics from async evaluations
naive_scores_async = naive_results_async['metrics'].scores
hybrid_scores_async = hybrid_results_async['metrics'].scores

# Compute averages for each metric
naive_df_async = pd.DataFrame(naive_scores_async)
hybrid_df_async = pd.DataFrame(hybrid_scores_async)

naive_averages_async = naive_df_async.mean()
hybrid_averages_async = hybrid_df_async.mean()

# Create comparison DataFrame
comparison_data_async = {
    'Metric': list(naive_averages_async.index),
    'Naive RAG': list(naive_averages_async.values),
    'Hybrid RAG': list(hybrid_averages_async.values)
}

comparison_df_async = pd.DataFrame(comparison_data_async)
comparison_df_async['Improvement (%)'] = (
    (comparison_df_async['Hybrid RAG'] - comparison_df_async['Naive RAG']) / 
    comparison_df_async['Naive RAG'] * 100
).round(2)
comparison_df_async['Better System'] = comparison_df_async.apply(
    lambda row: 'Hybrid RAG' if row['Hybrid RAG'] > row['Naive RAG'] 
    else 'Naive RAG' if row['Naive RAG'] > row['Hybrid RAG'] 
    else 'Tie', axis=1
)

# Round scores
comparison_df_async['Naive RAG'] = comparison_df_async['Naive RAG'].round(4)
comparison_df_async['Hybrid RAG'] = comparison_df_async['Hybrid RAG'].round(4)

print("\n" + "=" * 80)
print("COMPREHENSIVE RAG SYSTEM COMPARISON (ASYNC EVALUATION)")
print("=" * 80)
print(comparison_df_async.to_string(index=False))
print("\n" + "=" * 80)

# Calculate overall winner
hybrid_wins = sum(comparison_df_async['Hybrid RAG'] > comparison_df_async['Naive RAG'])
naive_wins = sum(comparison_df_async['Naive RAG'] > comparison_df_async['Hybrid RAG'])
ties = sum(comparison_df_async['Naive RAG'] == comparison_df_async['Hybrid RAG'])

print(f"\nFINAL SCORECARD:")
print(f"Hybrid RAG wins: {hybrid_wins} metrics")
print(f"Naive RAG wins: {naive_wins} metrics") 
print(f"Ties: {ties} metrics")

if hybrid_wins > naive_wins:
    print(f"\nOVERALL WINNER: Hybrid RAG SuperComponent!")
    print(f"   Better performance in {hybrid_wins}/{len(comparison_df_async)} metrics")
elif naive_wins > hybrid_wins:
    print(f"\nOVERALL WINNER: Naive RAG SuperComponent!")  
    print(f"   Better performance in {naive_wins}/{len(comparison_df_async)} metrics")
else:
    print(f"\nRESULT: It's a tie between both systems!")
    
avg_improvement = comparison_df_async['Improvement (%)'].mean()
print(f"\nAverage improvement by Hybrid RAG: {avg_improvement:.2f}%")


COMPREHENSIVE RAG SYSTEM COMPARISON (ASYNC EVALUATION)
                      Metric  Naive RAG  Hybrid RAG  Improvement (%) Better System
                faithfulness     0.8972      0.8636            -3.74     Naive RAG
            answer_relevancy     0.8696      0.8679            -0.20     Naive RAG
              context_recall     0.9100      0.9267             1.83    Hybrid RAG
factual_correctness(mode=f1)     0.5140      0.5940            15.56    Hybrid RAG


FINAL SCORECARD:
Hybrid RAG wins: 2 metrics
Naive RAG wins: 2 metrics
Ties: 0 metrics

RESULT: It's a tie between both systems!

Average improvement by Hybrid RAG: 3.36%
